In [1]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import ccxt
import time
from math import isclose
import Metriche_di_performance as mdp
from datetime import datetime

In [2]:
def download_and_plot(cryptos, timeframe, start_date, end_date, plot):
    
    exchange = ccxt.binance()  # Usa Binance come exchange
    exchange.load_markets()  # Carica i mercati disponibili
    cryptos_pairs = [f"{crypto}/USDT" for crypto in cryptos]  # Adatta i simboli per Binance

    # Converti date in timestamp
    start_timestamp = int(pd.Timestamp(start_date).timestamp() * 1000)
    end_timestamp = int(pd.Timestamp(end_date).timestamp() * 1000)

    data_frames = {}

    for pair in cryptos_pairs:
        if pair not in exchange.markets:
            raise ValueError(f"Il mercato {pair} non è disponibile su Binance.")

        print(f"Scarico i dati per {pair} dal {start_date} al {end_date}...")
        
        ohlcv = []
        since = start_timestamp

        # Scarica i dati in blocchi fino a raggiungere la data di fine
        while since < end_timestamp:
            batch = exchange.fetch_ohlcv(pair, timeframe, since)
            if not batch:
                break
            ohlcv.extend(batch)
            since = batch[-1][0] + 1  # Avanza al prossimo blocco
            time.sleep(exchange.rateLimit / 1000)  # Rispetta il rate limit dell'API

        # Crea un DataFrame dai dati
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)

        # Filtra i dati oltre la data di fine
        df = df[df.index <= pd.Timestamp(end_date)]
        data_frames[pair] = df

        if plot:
            # Plot dei dati
            plt.figure(figsize=(14, 8))

            # Grafico della prima crypto
            plt.subplot(3, 1, 1)
            plt.plot(data_frames[pair].index, data_frames[pair]['close'], label=f"{cryptos[0]} Price")
            plt.title(f"{pair} Price")
            plt.ylabel('Price (USDT)')
            plt.grid()
            plt.legend()

            plt.tight_layout()
            plt.show()

    return {k.replace('/USDT', ''): v for k, v in data_frames.items()}

In [3]:
def get_available_cryptos(exchange_name, timeframe, start_date, end_date):
    """
    Ottiene l'elenco delle crypto disponibili in un periodo specificato senza valori NaN.

    Parametri:
    - exchange_name: Nome dell'exchange (es. 'binance').
    - timeframe: Timeframe scelto (es. '1d', '1h').
    - start_date: Data di inizio (stringa 'YYYY-MM-DD').
    - end_date: Data di fine (stringa 'YYYY-MM-DD').

    Ritorna:
    - Lista di simboli crypto disponibili.
    """
    # Inizializza l'exchange CCXT
    exchange = getattr(ccxt, exchange_name)()

    # Converte le date in timestamp UNIX (ms)
    since = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp() * 1000)
    until = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp() * 1000)

    # Ottieni l'elenco dei mercati disponibili
    markets = exchange.load_markets()
    symbols = [market for market in markets.keys()]

    # Lista per salvare le crypto disponibili
    available_cryptos = []

    for symbol in symbols:
        try:
            # Scarica i dati OHLCV
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit=None)
            
            # Converte i dati in DataFrame
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            
            # Filtra per il periodo specificato
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df = df[(df['timestamp'] >= start_date) & (df['timestamp'] <= end_date)]
            
            # Verifica se ci sono valori NaN
            if not df.isnull().values.any():
                print(f'Trovato il pair {symbol}')
                available_cryptos.append(symbol)
            else:
                print(f'Il pair {symbol} ha dei NaN')
        
        except Exception as e:
            # Ignora gli errori (ad esempio mercati non supportati nel periodo scelto)
            print(f"Errore con il simbolo {symbol}: {e}")

    return available_cryptos

In [4]:
# Esempio di utilizzo
exchange_name = 'bybit'  # Nome dell'exchange
timeframe = '1d'  # Timeframe (es. '1d' per giornaliero)
start_date = '2021-01-01'  # Data di inizio
end_date = '2021-12-31'  # Data di fine

available_cryptos = get_available_cryptos(exchange_name, timeframe, start_date, end_date)
print("Crypto disponibili:", available_cryptos)

Trovato il pair BTC/USDT
Trovato il pair ETH/USDT
Trovato il pair XRP/USDT
Trovato il pair EOS/USDT
Trovato il pair ETH/BTC
Trovato il pair XRP/BTC
Trovato il pair DOT/USDT
Trovato il pair XLM/USDT
Trovato il pair LTC/USDT
Trovato il pair DOGE/USDT
Trovato il pair CHZ/USDT
Trovato il pair AXS/USDT
Trovato il pair MANA/USDT
Trovato il pair DYDX/USDT
Trovato il pair MKR/USDT
Trovato il pair COMP/USDT
Trovato il pair AAVE/USDT
Trovato il pair YFI/USDT
Trovato il pair LINK/USDT
Trovato il pair SUSHI/USDT
Trovato il pair UNI/USDT
Trovato il pair KSM/USDT
Trovato il pair ICP/USDT
Trovato il pair ADA/USDT
Trovato il pair ETC/USDT
Trovato il pair XTZ/USDT
Trovato il pair BCH/USDT
Trovato il pair QNT/USDT
Trovato il pair USDC/USDT
Trovato il pair GRT/USDT
Trovato il pair SOL/USDT
Trovato il pair FIL/USDT
Trovato il pair OMG/USDT
Trovato il pair BAT/USDT
Trovato il pair ZRX/USDT
Trovato il pair CRV/USDT
Trovato il pair AGLD/USDT
Trovato il pair ANKR/USDT
Trovato il pair PERP/USDT
Trovato il pair

In [5]:
len(available_cryptos)

1146

In [17]:
available_cryptos_usdt = [symbol.split('/USDT')[0] for symbol in available_cryptos if symbol.endswith('/USDT')]
len(available_cryptos_usdt)

517

In [18]:
available_cryptos_usdt

['BTC',
 'ETH',
 'XRP',
 'EOS',
 'DOT',
 'XLM',
 'LTC',
 'DOGE',
 'CHZ',
 'AXS',
 'MANA',
 'DYDX',
 'MKR',
 'COMP',
 'AAVE',
 'YFI',
 'LINK',
 'SUSHI',
 'UNI',
 'KSM',
 'ICP',
 'ADA',
 'ETC',
 'XTZ',
 'BCH',
 'QNT',
 'USDC',
 'GRT',
 'SOL',
 'FIL',
 'OMG',
 'BAT',
 'ZRX',
 'CRV',
 'AGLD',
 'ANKR',
 'PERP',
 'WAVES',
 'LUNC',
 'SPELL',
 'SHIB',
 'FTM',
 'ATOM',
 'ALGO',
 'ENJ',
 'CBX',
 'SAND',
 'AVAX',
 'WOO',
 'FTT',
 'GODS',
 'IMX',
 'ENS',
 'CAKE',
 'STETH',
 'GALFT',
 'SLP',
 'C98',
 'PSP',
 'GENE',
 'AVA',
 'ONE',
 'PTU',
 'XYM',
 'BOBA',
 'JASMY',
 'GALA',
 'TRVL',
 'WEMIX',
 'XEM',
 'BICO',
 'CEL',
 'UMA',
 'HOT',
 'NEXO',
 'BNT',
 'SNX',
 'REN',
 '1INCH',
 'TEL',
 'SIS',
 'LRC',
 'LDO',
 'REAL',
 '1SOL',
 'PLT',
 'IZI',
 'QTUM',
 'ZEN',
 'THETA',
 'MX',
 'DGB',
 'RVN',
 'EGLD',
 'RUNE',
 'RAIN',
 'XEC',
 'ICX',
 'XDC',
 'HNT',
 'BTG',
 'ZIL',
 'HBAR',
 'FLOW',
 'KASTA',
 'STX',
 'SIDUS',
 'VPAD',
 'LOOKS',
 'MBS',
 'DAI',
 'ACA',
 'MV',
 'MIX',
 'RSS3',
 'TAP',
 'ERTHA',
 'GMX'

In [ ]:
chosen_cryptos = ['BTC', 'ETH', 'SOL', 'LINK', 'FET', 'DOGE'] #available_cryptos_usdt[:]
timeframe_download = '1d'

data_frames = download_and_plot(chosen_cryptos, timeframe_download, start_date, end_date, plot=False)

# Prepara il DataFrame `crypto_prices` con timestamp e prezzi di chiusura
crypto_prices = pd.DataFrame()
crypto_prices['timestamp'] = data_frames[chosen_cryptos[0]].index
for crypto in chosen_cryptos:
    # print(crypto)
    # display(data_frames[crypto]['close'].values)
    crypto_prices[crypto] = data_frames[crypto]['close'].values
crypto_prices.set_index('timestamp', inplace=True)

Scarico i dati per BTC/USDT dal 2021-01-01 al 2021-12-31...
Scarico i dati per ETH/USDT dal 2021-01-01 al 2021-12-31...
Scarico i dati per SOL/USDT dal 2021-01-01 al 2021-12-31...
Scarico i dati per LINK/USDT dal 2021-01-01 al 2021-12-31...
Scarico i dati per FET/USDT dal 2021-01-01 al 2021-12-31...
Scarico i dati per DOGE/USDT dal 2021-01-01 al 2021-12-31...
BTC


array([29331.69, 32178.33, 33000.05, 31988.71, 33949.53, 36769.36,
       39432.28, 40582.81, 40088.22, 38150.02, 35404.47, 34051.24,
       37371.38, 39144.5 , 36742.22, 35994.98, 35828.61, 36631.27,
       35891.49, 35468.23, 30850.13, 32945.17, 32078.  , 32259.9 ,
       32254.2 , 32467.77, 30366.15, 33364.86, 34252.2 , 34262.88,
       33092.98, 33526.37, 35466.24, 37618.87, 36936.66, 38290.24,
       39186.94, 38795.69, 46374.87, 46420.42, 44807.58, 47969.51,
       47287.6 , 47153.69, 48577.79, 47911.1 , 49133.45, 52119.71,
       51552.6 , 55906.  , 55841.19, 57408.57, 54087.67, 48891.  ,
       49676.2 , 47073.73, 46276.87, 46106.43, 45135.66, 49587.03,
       48440.65, 50349.37, 48374.09, 48751.71, 48882.2 , 50971.75,
       52375.17, 54884.5 , 55851.59, 57773.16, 57221.72, 61188.39,
       58968.31, 55605.2 , 56900.75, 58912.97, 57648.16, 58030.01,
       58102.28, 57351.56, 54083.25, 54340.89, 52303.65, 51293.78,
       55025.59, 55817.14, 55777.63, 57635.47, 58746.57, 58740

ETH


array([ 728.91,  774.56,  978.28, 1041.43, 1099.56, 1208.42, 1224.87,
       1216.93, 1276.  , 1254.25, 1087.01, 1050.16, 1129.89, 1231.62,
       1168.53, 1227.83, 1232.45, 1258.45, 1365.36, 1376.99, 1110.96,
       1232.62, 1233.6 , 1392.51, 1317.88, 1365.97, 1239.03, 1329.69,
       1379.5 , 1378.05, 1312.55, 1374.17, 1511.87, 1664.49, 1595.67,
       1719.19, 1676.35, 1611.62, 1750.15, 1769.13, 1740.78, 1785.62,
       1840.26, 1814.96, 1800.42, 1779.25, 1781.99, 1849.88, 1939.61,
       1955.  , 1913.  , 1933.53, 1777.58, 1577.89, 1623.39, 1481.75,
       1444.59, 1458.13, 1419.18, 1569.86, 1486.78, 1567.69, 1539.23,
       1528.31, 1650.35, 1726.16, 1833.18, 1870.51, 1795.12, 1825.78,
       1766.1 , 1922.35, 1848.1 , 1793.48, 1805.12, 1823.58, 1776.08,
       1809.1 , 1806.1 , 1782.98, 1680.97, 1668.08, 1581.84, 1585.74,
       1698.84, 1712.63, 1687.01, 1816.74, 1840.46, 1919.37, 1968.05,
       2133.69, 2008.52, 2075.69, 2107.38, 2112.21, 1963.47, 2080.46,
       2066.87, 2133

SOL


...

array([11.8477, 12.2215, 13.6814, 13.5935, 14.5162, 17.2322, 15.9628,
       15.2381, 17.5501, 16.206 , 14.6393, 13.9655, 15.9356, 17.9804,
       20.7375, 20.1054, 23.2819, 22.0336, 20.5557, 21.8482, 18.4119,
       21.5519, 24.8046, 24.7935, 23.3418, 23.0981, 20.9586, 23.0145,
       22.748 , 23.6399, 22.5613, 22.8757, 23.6582, 25.087 , 24.492 ,
       26.3034, 25.0231, 24.7254, 25.4115, 27.6034, 26.8121, 27.8475,
       30.6265, 34.1941, 33.1565, 32.5899, 31.9632, 32.2324, 32.4801,
       34.6846, 33.9387, 34.0915, 31.4542, 25.9249, 28.0851, 24.7144,
       25.1862, 26.1896, 24.7016, 27.7116, 28.4769, 29.85  , 27.6943,
       27.8343, 28.1479, 28.5932, 31.8469, 31.4914, 29.9036, 30.1166,
       28.3553, 29.7384, 28.5469, 27.5091, 27.9099, 31.0579, 29.3949,
       29.777 , 29.6465, 29.2097, 27.2431, 26.7813, 24.8684, 25.535 ,
       27.3927, 26.8766, 26.6588, 28.0357, 27.8301, 29.4167, 30.0684,
       32.4864, 29.293 , 30.7471, 32.3791, 34.725 , 31.109 , 32.7884,
       31.5893, 31.8

FET


array([0.05333, 0.05127, 0.05671, 0.05723, 0.06182, 0.07311, 0.07112,
       0.07104, 0.07261, 0.0699 , 0.06049, 0.05881, 0.06604, 0.07696,
       0.08851, 0.08459, 0.09487, 0.0968 , 0.09115, 0.092  , 0.08064,
       0.08955, 0.08933, 0.08779, 0.08253, 0.09518, 0.09   , 0.08857,
       0.11164, 0.10452, 0.09803, 0.10884, 0.1101 , 0.15841, 0.14849,
       0.17848, 0.17   , 0.18214, 0.21432, 0.25112, 0.23952, 0.23532,
       0.25158, 0.26402, 0.24813, 0.2394 , 0.22679, 0.23594, 0.30729,
       0.29228, 0.2884 , 0.29609, 0.27521, 0.22399, 0.23287, 0.22555,
       0.22514, 0.27529, 0.24682, 0.27925, 0.31185, 0.36388, 0.32349,
       0.3737 , 0.3831 , 0.37246, 0.44438, 0.42838, 0.454  , 0.44166,
       0.5255 , 0.62092, 0.68704, 0.6461 , 0.61549, 0.658  , 0.59455,
       0.6038 , 0.58633, 0.64182, 0.67196, 0.62812, 0.61858, 0.60863,
       0.69757, 0.78885, 0.77916, 0.82   , 0.80995, 0.72674, 0.70796,
       0.69364, 0.63403, 0.72304, 0.68135, 0.66701, 0.59566, 0.6066 ,
       0.68365, 0.65

DOGE


array([0.00568  , 0.0105264, 0.0098209, 0.0097613, 0.00997  , 0.01042  ,
       0.0097439, 0.00974  , 0.0101789, 0.00982  , 0.0088326, 0.0080807,
       0.0086738, 0.0093907, 0.0093525, 0.0092374, 0.0090987, 0.0091782,
       0.0090428, 0.0090487, 0.0081581, 0.0085075, 0.0085779, 0.0087135,
       0.0083509, 0.0082472, 0.0074124, 0.0364995, 0.0459059, 0.02826  ,
       0.037    , 0.0348173, 0.0313489, 0.0372597, 0.0529081, 0.0467193,
       0.0576559, 0.0779247, 0.0787   , 0.0700121, 0.0725527, 0.069583 ,
       0.0697675, 0.0662405, 0.0626783, 0.0565   , 0.053476 , 0.0495187,
       0.0587991, 0.055033 , 0.0541708, 0.0560183, 0.0535557, 0.0476562,
       0.0567911, 0.0499138, 0.0504768, 0.0500725, 0.0481582, 0.0505933,
       0.050364 , 0.0504146, 0.0499865, 0.0495285, 0.0509588, 0.0519871,
       0.0629738, 0.0579117, 0.0558692, 0.0559286, 0.0552731, 0.0627892,
       0.058221 , 0.0569691, 0.0586507, 0.057892 , 0.0573485, 0.0582472,
       0.0588186, 0.0574781, 0.054888 , 0.0533532, 

# Analisi statistiche

## Statistiche principali

In [ ]:
def analyze_crypto_prices(crypto_prices, start_date, end_date):
    # Filtra il periodo selezionato
    filtered_data = crypto_prices.loc[start_date:end_date]
    
    # Statistiche principali: media, minimo, massimo, std e percentili
    stats = filtered_data.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]).T
    stats.rename(columns={"50%": "median"}, inplace=True)
    stats['range'] = stats['max'] - stats['min']
    
    # Calcola variazioni giornaliere (delta price)
    daily_changes = filtered_data.pct_change().dropna()
    daily_stats = daily_changes.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]).T
    daily_stats.rename(columns={"50%": "median"}, inplace=True)
    daily_stats['range'] = daily_stats['max'] - daily_stats['min']

    return stats, daily_stats, filtered_data, daily_changes

## Analisi salite e discese

In [36]:
def analyze_trends(filtered_data, timeframe_download):
    # Mappa del timeframe in unità di tempo
    timeframe_to_seconds = {
        '1m': 60,       # 1 minuto
        '5m': 300,      # 5 minuti
        '15m': 900,     # 15 minuti
        '30m': 1800,    # 30 minuti
        '1h': 3600,     # 1 ora
        '4h': 14400,    # 4 ore
        '1d': 86400,    # 1 giorno
        '1w': 604800    # 1 settimana
    }
    
    # Ottieni i secondi per il timeframe scelto
    timeframe_seconds = timeframe_to_seconds.get(timeframe_download)
    if not timeframe_seconds:
        raise ValueError(f"Timeframe '{timeframe_download}' non supportato. Aggiorna la mappa.")

    results = {}
    for crypto in filtered_data.columns:
        # Calcola variazioni giornaliere
        changes = filtered_data[crypto].diff()
        
        # Identifica salite e discese
        rises = changes[changes > 0]
        falls = changes[changes < 0]

        # Calcola durate come differenze di indici in termini di timeframe scelto
        rise_durations = rises.index.to_series().diff().dt.total_seconds() / timeframe_seconds
        fall_durations = falls.index.to_series().diff().dt.total_seconds() / timeframe_seconds

        # Statistiche per ogni crypto
        results[crypto] = {
            'num_rises': len(rises),
            'num_falls': len(falls),
            'avg_rise_pct': (rises / filtered_data[crypto].shift())[rises.index].mean() * 100,
            'avg_fall_pct': (falls / filtered_data[crypto].shift())[falls.index].mean() * 100,
            'avg_rise_duration': rise_durations.mean(),
            'avg_fall_duration': fall_durations.mean(),
        }
    
    # Riassunto generale
    summary = pd.DataFrame(results).T
    overall_summary = {
        'mean_rise_pct': summary['avg_rise_pct'].mean(),
        'mean_fall_pct': summary['avg_fall_pct'].mean(),
        'mean_rise_duration': summary['avg_rise_duration'].mean(),
        'mean_fall_duration': summary['avg_fall_duration'].mean(),
        'max_rise_pct': summary['avg_rise_pct'].max(),
        'min_rise_pct': summary['avg_rise_pct'].min(),
        'max_fall_pct': summary['avg_fall_pct'].max(),
        'min_fall_pct': summary['avg_fall_pct'].min(),
    }
    return summary, overall_summary

## Analisi correlazioni

In [43]:
def analyze_correlations(filtered_data):
    # Calcola la matrice di correlazione
    correlation_matrix = filtered_data.corr()
    
    # Trasforma la matrice in un Series con coppie come indice
    correlation_pairs = correlation_matrix.unstack()

    # Rimuovi duplicati e auto-correlazioni (crypto con se stessa)
    correlation_pairs = correlation_pairs[correlation_pairs.index.get_level_values(0) != correlation_pairs.index.get_level_values(1)]

    # Trova le coppie di crypto più e meno correlate
    most_correlated = correlation_pairs.idxmax()
    least_correlated = correlation_pairs.idxmin()

    # Correlazione media per crypto
    mean_correlations = correlation_matrix.mean()

    return correlation_matrix, most_correlated, least_correlated, mean_correlations

## Output completo

In [44]:
def full_crypto_analysis(crypto_prices, start_date, end_date):
    # 1. Statistiche principali e variazioni giornaliere
    stats, daily_stats, filtered_data, daily_changes = analyze_crypto_prices(crypto_prices, start_date, end_date)
    
    # 2. Salite e discese
    trend_summary, overall_trends = analyze_trends(filtered_data, timeframe_download)
    
    # 3. Correlazioni
    corr_matrix, most_corr, least_corr, mean_corr = analyze_correlations(filtered_data)
    
    # Output dei risultati
    print("=== Statistiche principali sui prezzi ===")
    display(stats)
    
    print("\n=== Statistiche delle variazioni giornaliere ===")
    display(daily_stats)
    
    print("\n=== Analisi di salite e discese ===")
    display(trend_summary)
    print("\nRiassunto generale:")
    display(overall_trends)
    
    print("\n=== Matrice di correlazione ===")
    display(corr_matrix)
    print("\nCripto più correlate:", most_corr)
    print("Cripto meno correlate:", least_corr)
    print("\nCorrelazione media per crypto:")
    display(mean_corr)

In [45]:
# Esegui l'analisi completa
full_crypto_analysis(crypto_prices, start_date, end_date)

=== Statistiche principali sui prezzi ===


,count,mean,std,min,10%,25%,median,75%,90%,max,range
BTC,365.0,47400.003041,9770.992384,29331.69000,33728.704000,38290.240000,47800.00000,56150.0100,60220.834000,67525.83000,38194.14000
ETH,365.0,2777.428740,1025.256532,728.91000,1496.816000,1891.640000,2634.57000,3646.6800,4204.432000,4807.98000,4079.07000
SOL,365.0,80.098492,77.115613,1.79990,6.698760,19.750100,38.92500,157.5400,197.786000,258.44000,256.64010
LINK,365.0,26.626388,7.026862,11.84770,18.259600,22.062000,26.39000,30.1900,34.687840,52.26900,40.42130
FET,365.0,0.489323,0.242670,0.05127,0.179944,0.279250,0.50178,0.7106,0.806690,0.94970,0.89843
DOGE,365.0,0.204595,0.126550,0.00568,0.048702,0.063717,0.21850,0.2720,0.330656,0.68982,0.68414



=== Statistiche delle variazioni giornaliere ===


,count,mean,std,min,10%,25%,median,75%,90%,max,range
BTC,364.0,0.002148,0.042502,-0.143751,-0.047509,-0.021515,0.000260,0.026639,0.054049,0.195361,0.339112
ETH,364.0,0.006066,0.056753,-0.277372,-0.055983,-0.024959,0.005793,0.037474,0.073249,0.263329,0.540701
SOL,364.0,0.016044,0.085020,-0.374958,-0.071743,-0.033587,0.007036,0.052088,0.133742,0.362979,0.737936
LINK,364.0,0.004126,0.073747,-0.383851,-0.085282,-0.038074,0.006097,0.048358,0.089698,0.330982,0.714833
FET,364.0,0.010341,0.092936,-0.356014,-0.095631,-0.042387,-0.000972,0.055830,0.129704,0.438783,0.794797
DOGE,364.0,0.021629,0.236421,-0.384393,-0.075385,-0.038837,-0.003254,0.028781,0.132374,3.924114,4.308507



=== Analisi di salite e discese ===


...

{'mean_rise_pct': np.float64(6.389836723722663),
 'mean_fall_pct': np.float64(-4.740129337456636),
 'mean_rise_duration': np.float64(1.9287869547503618),
 'mean_fall_duration': np.float64(2.0877540984276606),
 'max_rise_pct': np.float64(10.175230492760038),
 'min_rise_pct': np.float64(3.3484716389249143),
 'max_fall_pct': np.float64(-2.9535415830227274),
 'min_fall_pct': np.float64(-5.670563166959399)}


=== Matrice di correlazione ===


,BTC,ETH,SOL,LINK,FET,DOGE
BTC,1.000000,0.556974,0.506112,0.645839,0.772458,0.120737
ETH,0.556974,1.000000,0.896339,0.301835,0.746426,0.581807
SOL,0.506112,0.896339,1.000000,0.052633,0.752038,0.274465
LINK,0.645839,0.301835,0.052633,1.000000,0.433358,0.476607
FET,0.772458,0.746426,0.752038,0.433358,1.000000,0.308881
DOGE,0.120737,0.581807,0.274465,0.476607,0.308881,1.000000



Cripto più correlate: ('ETH', 'SOL')
Cripto meno correlate: ('SOL', 'LINK')

Correlazione media per crypto:


BTC     0.600353
ETH     0.680563
SOL     0.580265
LINK    0.485045
FET     0.668860
DOGE    0.460416
dtype: float64